# Run Algonauts Challenge 2023 Pipeline

Here you can run the challenge pipeline (ie. generate predictions) using a pretrained model from keras.applications, or load a model from a model file.
This notebook will output challenge submission files, as well as correlation graphs and a results.json file
containing all correlation results for all layers, subjects and ROIs.
You can also print best layers per ROI and generate submission files merged from the best layers in addition
to predictions from single layers.

In [ ]:
import src.algonauts.data_processors.image_transforms
from src.algonauts.models import model_loaders

Define experiment name here. Output files will be generated under `{base_folder}/data/out/{experiment}`

In [ ]:
experiment = 'alexnet_coco'
batch_size = 300

Pick environment here. If running locally, pick 'jupyter_notebook'.

In [ ]:
platform = 'jupyter_notebook' #@param ['colab', 'paperspace'] {allow-input: true}

if platform == 'jupyter_notebook':
    data_dir = '../data'
    challenge_data_dir = f'{data_dir}/algonauts_2023_challenge_data'
    exp_output_dir = f'{data_dir}/out/{experiment}'

if platform == 'paperspace':
    data_dir = '.'
    challenge_data_dir = '/storage/algonauts_2023_challenge_data'
    exp_output_dir = f'/notebooks/data/out/{experiment}'

if platform == 'colab':
    data_dir = '/content/drive/MyDrive/algonauts23/data'
    challenge_data_dir = f'{data_dir}/algonauts_2023_challenge_data'
    exp_output_dir = f'{data_dir}/out/{experiment}'

## Specify model and appropriate image preprocessing steps
Define a function to load and return the model, printing its layers.
Here we first load the model and print its nodes

### Choose to load a pretrained model from keras.applications

In [ ]:
model_loader = lambda: model_loaders.load_vgg16()

### Or choose to load a model from file

We use alexnet_coco as an example. Change model filename to load a different model. You must also pick or
implement an image_transform for the model's needs that will be used on the stimulus images.

In [ ]:
model_filename = f'{data_dir}/models/alexnet_coco.h5'
transform_image = src.algonauts.data_processors.image_transforms.transform_alexnet
model_loader = lambda: model_loaders.load_from_file(model_filename, transform_image)

### Print the model layers

In [ ]:
model, _ = model_loader()
print(*(layer.name for layer in model.layers), sep=' -> ')
del model

## Select layers and subjects
Now let's define which layer(s) we will pick from, and which subject(s) to process

In [ ]:
layers = ['block1_pool', 'block2_pool', 'block3_pool', 'block4_pool', 'block5_pool']
subjects = [
    1, 2, 3, 4, 5, 6, 7, 8
    ]


## Run pipeline

In [ ]:
from src.algonauts.pipelines.tf_pipeline import run_tf_pipeline

run_tf_pipeline(batch_size=batch_size, model_loader=model_loader, layers=layers, subjects=subjects,
                challenge_data_dir=challenge_data_dir,
                exp_output_dir=exp_output_dir)

## Merge predictions from best layers for each ROI

In [ ]:
from src.algonauts.evaluators.predictions import merge_predictions_for_all_subjects
merge_predictions_for_all_subjects(subjects, challenge_data_dir, exp_output_dir)

## Print best layers

In [ ]:
import json
from src.algonauts.evaluators.correlations import find_best_correlations

subj = 1
result = find_best_correlations(f'{exp_output_dir}/results.json', subj)
print(json.dumps(result, indent=2))
